In [ ]:
!pip install stanza
import stanza
import sqlite3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 45.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [ ]:
con = sqlite3.connect("proyectoPura.db")
cur = con.cursor()

query = """
SELECT Título_Obra FROM OBRAS
UNION ALL
SELECT Título_Complemento FROM COMPLEMENTOS
UNION ALL
SELECT Título_Sección FROM SECCIONES_PUBLICADAS
"""

cur.execute(query)
titles = cur.fetchall()
#print(titles)

titles_list = []
for title in titles:
  if title[0] is not None:
    titles_list.append(title[0])
  if title[0] is None:
    titles_list.append("NULL") #we take into account "NULL" titles, to keep the same structure as original DB.
print(titles_list)

con.close()

['Al Sr. D. R. P. Enviándole mis obras', 'La mujer piadosa (Fragmentos)', 'Desvarío', 'Soneto. "¡Gloria y pobreza! El inmortal Quintana..."', 'Balada. La despedida de una hija', 'Un viaje a la Luna. Prólogo: La pesadilla de Mocquet', 'En el álbum de la señora M…', 'La primavera', 'La perejilera', 'Un viaje a la Luna. (Continuación)', 'Al señor Don B. M.', 'A orillas del Manzanares. La súplica', 'Un viaje a la Luna. (Continuación)', 'Fragmentos de la Rosa de Alejandra', 'Gente morena', 'Un viaje a la Luna. (Continuación)', 'Al excmo. Sr. D. Agustín Esteban Collantes, en el día de su santo', 'A la niña Margarita Aurora de Wilson y Serrano, el día de su cumpleaños', 'Costumbres del siglo XIX. I', 'Margarita. Novela bretona', 'Los escondidos y la tapada. Fragmentos', 'A mi apreciable amiga la Señora doña M. Del Pilar Sinués de Marco en su álbum', 'Costumbres del siglo XIX.II', 'Margarita. Novela bretona. (Continuación)', 'A la niña Margarita Aurora de Wilson y Serrano', 'A María', 'A su Al

In [ ]:
#we start complete formatting before NER:

import re

def subtitle_format(matchobj):
  match = matchobj.group(0)
  if re.fullmatch(r'\s*"\s*-\s*', match):
    return '" | '
  elif re.fullmatch(r'\s*-\s*"\s*', match):
    return ' | "'
  elif re.fullmatch(r'\s*-\s*"\s*-\s*', match):
    return '" | '
  elif re.fullmatch(r'\s*\[\.\.\.\]\s*"\s*-\s*', match):
    submatch = re.fullmatch(r'(\s*)\[\.\.\.\]\s*"\s*-\s*', match)
    return submatch.group(1)+'[...]" | '

def formatting_subtitles(text):
    subtit_re = r'\s*"\s*-\s*|\s*-\s*"\s*|\s*-\s*"\s*-\s*|\s*\[\.\.\.\]\s*"\s*-\s*'
    new_text = re.sub(subtit_re, subtitle_format, text)
    return new_text

def formatting_century(text):
  re_century_1 = r'(siglo|siglos)\s*(I|II|III|IV|V|VI|VII|VIII|IX|X|XI|XII|XIII|XIV|XV|XVI|XVII|XVIII|XIX|XX|XXI)\.\s*'
  re_century_2 = r'(siglo|siglos)\s*(I|II|III|IV|V|VI|VII|VIII|IX|X|XI|XII|XIII|XIV|XV|XVI|XVII|XVIII|XIX|XX|XXI)\s+\.'
  if re.search(re_century_1, text):
    return re.sub(re_century_1, r'[("\1 \2", "DATE")] ', text).strip()
  if re.search(re_century_2, text):
    return re.sub(re_century_2, r'[("\1 \2", "DATE")]', text).strip()
  else:
    return text

encontrar_subtit = [formatting_subtitles(row) for row in titles_list]
list_titsub = [row for row in encontrar_subtit if " | " in row]

#only this exceptions do not adjust to pattern (2 rows):
excepcion_1 = """Piquer-Oda "Alzad vuestras"""
excepcion_2 = """Matrimonios por anuncios-Los baños de Biarritz"""
def subadhoc(matchobj):
  return ' "'+ matchobj.group(0)[1:]
for i, row in enumerate(encontrar_subtit):
  if excepcion_1 in row:
    encontrar_subtit[i] = row.replace('Piquer-Oda "Alzad', 'Piquer-Oda. "Alzad') #Mismo formato que algunos sonetos.
  if excepcion_2 in row:
    encontrar_subtit[i]= row.replace('-', ' | ')
    #encontrar_subtit[i] = re.sub(r'[^|]+\]"', subadhoc, encontrar_subtit[i])##N: añadido mío (con la definición de subadhoc)
#print(encontrar_subtit[249])
#print(encontrar_subtit[207])

encontrar_century = [formatting_century(row).strip() for row in encontrar_subtit]

encontrar_century[:30]

['Al Sr. D. R. P. Enviándole mis obras',
 'La mujer piadosa (Fragmentos)',
 'Desvarío',
 'Soneto. "¡Gloria y pobreza! El inmortal Quintana..."',
 'Balada. La despedida de una hija',
 'Un viaje a la Luna. Prólogo: La pesadilla de Mocquet',
 'En el álbum de la señora M…',
 'La primavera',
 'La perejilera',
 'Un viaje a la Luna. (Continuación)',
 'Al señor Don B. M.',
 'A orillas del Manzanares. La súplica',
 'Un viaje a la Luna. (Continuación)',
 'Fragmentos de la Rosa de Alejandra',
 'Gente morena',
 'Un viaje a la Luna. (Continuación)',
 'Al excmo. Sr. D. Agustín Esteban Collantes, en el día de su santo',
 'A la niña Margarita Aurora de Wilson y Serrano, el día de su cumpleaños',
 'Costumbres del [("siglo XIX", "DATE")] I',
 'Margarita. Novela bretona',
 'Los escondidos y la tapada. Fragmentos',
 'A mi apreciable amiga la Señora doña M. Del Pilar Sinués de Marco en su álbum',
 'Costumbres del [("siglo XIX", "DATE")] II',
 'Margarita. Novela bretona. (Continuación)',
 'A la niña Margari

In [ ]:
#we continue preparing each title to process it through model (ner)

#def clean_per_title(title):
  #cleaned_text = re.sub(
    #r'\s*\(\s*(Continuación|Fragmentos?|Conclusión|Conclusiones?|Conclusion)\.?\s*\)\s*', #not now, mayber after processing
    #' ',
    #title)
  #cleaned_text = re.sub(r'\s*"\s*|\s*\[\s*\.\s*\.\s*\.\s*\]\s*|\s*\[\s*\u2026\s*\]\s*', ' ', cleaned_text)
  #return cleaned_text.strip()

#cleaned_rows = [clean_per_title(row) for row in encontrar_subtit]

#print(cleaned_rows)
titles_to_NER = [row.split(" | ") if " | " in row else [row] for row in encontrar_century]

for row in titles_to_NER[:30]:
  print(row, "\n")

['Al Sr. D. R. P. Enviándole mis obras'] 

['La mujer piadosa (Fragmentos)'] 

['Desvarío'] 

['Soneto. "¡Gloria y pobreza! El inmortal Quintana..."'] 

['Balada. La despedida de una hija'] 

['Un viaje a la Luna. Prólogo: La pesadilla de Mocquet'] 

['En el álbum de la señora M…'] 

['La primavera'] 

['La perejilera'] 

['Un viaje a la Luna. (Continuación)'] 

['Al señor Don B. M.'] 

['A orillas del Manzanares. La súplica'] 

['Un viaje a la Luna. (Continuación)'] 

['Fragmentos de la Rosa de Alejandra'] 

['Gente morena'] 

['Un viaje a la Luna. (Continuación)'] 

['Al excmo. Sr. D. Agustín Esteban Collantes, en el día de su santo'] 

['A la niña Margarita Aurora de Wilson y Serrano, el día de su cumpleaños'] 

['Costumbres del [("siglo XIX", "DATE")] I'] 

['Margarita. Novela bretona'] 

['Los escondidos y la tapada. Fragmentos'] 

['A mi apreciable amiga la Señora doña M. Del Pilar Sinués de Marco en su álbum'] 

['Costumbres del [("siglo XIX", "DATE")] II'] 

['Margarita. Novela

In [ ]:
#first processing, we extract NER
def processing_1(text, nlp_model):
  doc = nlp_model(text)
  if text == "NULL":
    return [("NULL")]
  if "[(" not in text:
    return [(ent.text, ent.type) for ent in doc.entities]
  if "[(" in text:
    special_categories = re.findall(r'\[\(.*\)\]', text)
    parsing_text = re.split(r'\[\(.*\)\]', text)
    exception = []
    for part in parsing_text:
      if part.startswith("[(") and part.endswith(")]"):
        exception.append(part)
      else:
        doc = nlp_model(part)
        exception.extend([(ent.text, ent.type) for ent in doc.entities])
    return exception

nlp_es = stanza.Pipeline(lang="es", processors="tokenize,mwt,pos,lemma,ner,depparse")

processed_ent_es = [[processing_1(el, nlp_es) for el in lista] for lista in titles_to_NER]

for element in processed_ent_es:
  print(element, "\n")



INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json


INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |
| ner       | conll02           |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


[[('Sr. D. R. P.', 'MISC')]] 

[[('Fragmentos', 'MISC')]] 

[[('Desvarío', 'MISC')]] 

[[('Soneto', 'PER'), ('inmortal Quintana...', 'MISC')]] 

[[('Balada', 'PER')]] 

[[('Luna', 'MISC'), ('Mocquet', 'PER')]] 

[[('señora M…', 'PER')]] 

[[]] 

[[]] 

[[('Luna', 'MISC')]] 

[[('Don B. M.', 'PER')]] 

[[('Manzanares', 'LOC')]] 

[[('Luna', 'MISC')]] 

[[('de la Rosa de Alejandra', 'PER')]] 

[[]] 

[[('Luna', 'MISC')]] 

[[('Sr. D. Agustín Esteban Collantes', 'MISC')]] 

[[('Margarita Aurora de Wilson', 'PER'), ('Serrano', 'PER')]] 

[[('I', 'PER')]] 

[[('Margarita', 'PER')]] 

[[]] 

[[('Señora doña M. Del Pilar Sinués de Marco', 'PER')]] 

[[('II', 'MISC')]] 

[[('Margarita', 'PER')]] 

[[('Margarita Aurora de Wilson', 'PER'), ('Serrano', 'PER')]] 

[[('María', 'PER')]] 

[[('Alteza Real', 'PER'), ('Príncipe de Asturias', 'PER')]] 

[[('Margarita', 'PER')]] 

[[]] 

[[('Soneto', 'PER')]] 

[[('SS. MM. II', 'PER')]] 

[[('El matrimonio de la Princesa real de Inglaterra [...]', 'MISC'

In [ ]:
from ast import Pass
import copy

titles_to_NER_COPY = copy.deepcopy(titles_to_NER)

for index, row in enumerate(processed_ent_es):
  if index in [18, 22]:  # Problematic samples, we create exception
    continue
  else:
    for sub_index, entidad in enumerate(row):
      for tuple_item in entidad:
        if tuple_item is None:
          pass
        if isinstance(tuple_item, str):
          pass
        if isinstance(tuple_item, tuple):
          entity_str = tuple_item[0]
          replacement_str = f"[('{tuple_item[0]}', '{tuple_item[1]}')]"
          titles_to_NER_COPY[index][sub_index] = re.sub(re.escape(entity_str), replacement_str, titles_to_NER_COPY[index][sub_index])
        #except TypeError:
          #pass
        #except IndexError:
          #pass

for el in titles_to_NER_COPY[:50]:
    print(el, "\n")

["Al [('Sr. D. R. P.', 'MISC')] Enviándole mis obras"] 

["La mujer piadosa ([('Fragmentos', 'MISC')])"] 

["[('Desvarío', 'MISC')]"] 

['[(\'Soneto\', \'PER\')]. "¡Gloria y pobreza! El [(\'inmortal Quintana...\', \'MISC\')]"'] 

["[('Balada', 'PER')]. La despedida de una hija"] 

["Un viaje a la [('Luna', 'MISC')]. Prólogo: La pesadilla de [('Mocquet', 'PER')]"] 

["En el álbum de la [('señora M…', 'PER')]"] 

['La primavera'] 

['La perejilera'] 

["Un viaje a la [('Luna', 'MISC')]. (Continuación)"] 

["Al señor [('Don B. M.', 'PER')]"] 

["A orillas del [('Manzanares', 'LOC')]. La súplica"] 

["Un viaje a la [('Luna', 'MISC')]. (Continuación)"] 

["Fragmentos [('de la Rosa de Alejandra', 'PER')]"] 

['Gente morena'] 

["Un viaje a la [('Luna', 'MISC')]. (Continuación)"] 

["Al excmo. [('Sr. D. Agustín Esteban Collantes', 'MISC')], en el día de su santo"] 

["A la niña [('Margarita Aurora de Wilson', 'PER')] y [('Serrano', 'PER')], el día de su cumpleaños"] 

['Costumbres del [("sigl

In [ ]:
#we've gotten categories related to NER processing, after including entities [(x,y)] in each row/title
#we proceed with the second part of processing (tok,lemma,pos,depp) keeping NER categories without processing.

def processing_2(text, nlp_model):
  doc = nlp_model(text)
  if text == "NULL":
    return [("NULL")]
  if "[(" not in text:
    return [(word.text, word.lemma, word.upos) for sent in doc.sentences for word in sent.words]
  if "[(" in text:
    result = []
    current_index = 0
    for match in re.finditer(r'\[\(.*?\)\]', text):  # Find all special category patterns
      # Process text before the match
      before_match = text[current_index:match.start()]
      doc = nlp_model(before_match)
      result.extend([(word.text, word.lemma, word.upos) for sent in doc.sentences for word in sent.words])

      result.append(match.group(0))

      current_index = match.end()

    remaining_text = text[current_index:]
    doc = nlp_model(remaining_text)
    result.extend([(word.text, word.lemma, word.upos) for sent in doc.sentences for word in sent.words])

    return result

nlp_es = stanza.Pipeline(lang="es", processors="tokenize,mwt,pos,lemma")

pro_lem = [[processing_2(el, nlp_es) for el in lista] for lista in titles_to_NER_COPY]

for element in pro_lem[:50]:
  print(element, "\n")

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


[[('A', 'a', 'ADP'), ('el', 'el', 'DET'), "[('Sr. D. R. P.', 'MISC')]", ('Enviando', 'enviar', 'VERB'), ('le', 'él', 'PRON'), ('mis', 'mi', 'DET'), ('obras', 'obra', 'NOUN')]] 

[[('La', 'el', 'DET'), ('mujer', 'mujer', 'NOUN'), ('piadosa', 'piadoso', 'ADJ'), ('(', '(', 'PUNCT'), "[('Fragmentos', 'MISC')]", (')', ')', 'PUNCT')]] 

[["[('Desvarío', 'MISC')]"]] 

[["[('Soneto', 'PER')]", ('.', '.', 'PUNCT'), ('"', '"', 'PUNCT'), ('¡', '¡', 'PUNCT'), ('Gloria', 'gloria', 'PROPN'), ('y', 'y', 'CCONJ'), ('pobreza', 'pobreza', 'NOUN'), ('!', '!', 'PUNCT'), ('El', 'el', 'DET'), "[('inmortal Quintana...', 'MISC')]", ('"', '"', 'PUNCT')]] 

[["[('Balada', 'PER')]", ('.', '.', 'PUNCT'), ('La', 'el', 'DET'), ('despedida', 'despedida', 'NOUN'), ('de', 'de', 'ADP'), ('una', 'uno', 'DET'), ('hija', 'hija', 'NOUN')]] 

[[('Un', 'uno', 'DET'), ('viaje', 'viaje', 'NOUN'), ('a', 'a', 'ADP'), ('la', 'el', 'DET'), "[('Luna', 'MISC')]", ('.', '.', 'PUNCT'), ('Prólogo', 'prólogo', 'NOUN'), (':', ':', 'PUNCT

In [ ]:
pro_lem_cleaned = copy.deepcopy(pro_lem)

for i_element, element in enumerate(pro_lem):
  for i_titulo, titulo in enumerate(element):
    #pro_lem_cleaned[i_element][i_titulo] = [tupla for tupla in titulo if isinstance(tupla, tuple) and tupla[2] in ('NOUN', 'PROPN', 'ADJ', 'VERB', 'ADV', 'INTJ')]
      pro_lem_cleaned[i_element][i_titulo] = [tupla for tupla in titulo if isinstance(tupla, tuple) and tupla[2] in ('NOUN', 'PROPN', 'ADJ', 'VERB', 'ADV', 'INTJ') or isinstance(tupla, str)]

In [ ]:
for element in pro_lem_cleaned[:10]:
  print(element, "\n")

[["[('Sr. D. R. P.', 'MISC')]", ('Enviando', 'enviar', 'VERB'), ('obras', 'obra', 'NOUN')]] 

[[('mujer', 'mujer', 'NOUN'), ('piadosa', 'piadoso', 'ADJ'), "[('Fragmentos', 'MISC')]"]] 

[["[('Desvarío', 'MISC')]"]] 

[["[('Soneto', 'PER')]", ('Gloria', 'gloria', 'PROPN'), ('pobreza', 'pobreza', 'NOUN'), "[('inmortal Quintana...', 'MISC')]"]] 

[["[('Balada', 'PER')]", ('despedida', 'despedida', 'NOUN'), ('hija', 'hija', 'NOUN')]] 

[[('viaje', 'viaje', 'NOUN'), "[('Luna', 'MISC')]", ('Prólogo', 'prólogo', 'NOUN'), ('pesadilla', 'pesadilla', 'PROPN'), "[('Mocquet', 'PER')]"]] 

[[('álbum', 'álbum', 'NOUN'), "[('señora M…', 'PER')]"]] 

[[('primavera', 'primavera', 'NOUN')]] 

[[('perejilera', 'perejilera', 'NOUN')]] 

[[('viaje', 'viaje', 'NOUN'), "[('Luna', 'MISC')]", ('Continuación', 'continuación', 'NOUN')]] 



In [ ]:
with open("FINAL_CATEGORIES.txt", "a") as f:
  for element in pro_lem_cleaned:
    f.write(str(element)+"\n")

In [ ]:
#pro_lem_cleaned[:388]
import csv

with open("FINAL_CATEGORIES.csv", "w", newline="", encoding="utf-8") as f:
  writer = csv.writer(f)
  writer.writerow(["Categorías"])
  for element in pro_lem_cleaned[:388]:
    writer.writerow([str(element)])

In [ ]:
con = sqlite3.connect("ProyectoPura_Javier.db")
cur = con.cursor()

cur.execute("""
CREATE TABLE Categorías_RAW_Javier
(Categorías)
""")

with open("FINAL_CATEGORIES.csv","r", encoding="utf-8") as file:
  csv_reader = csv.reader(file)
  next(csv_reader) #ignoring head row
  for row in csv_reader:
    cur.execute("INSERT INTO Categorías_RAW_Javier (Categorías) VALUES (?)", row)

con.commit()
con.close()

In [ ]:
from collections import Counter

lema_freq = Counter()

for element in pro_lem_cleaned:
  for titulo_ in element:
    for cat in titulo_:
      if isinstance(cat, tuple):
        lema_freq.update([cat[1]])
        #lema_freq.update([cat[0]])

lema_freq.most_common()

[('revista', 65),
 ('moda', 39),
 ('mes', 19),
 ('casa', 18),
 ('histórico', 18),
 ('figurín', 16),
 ('explicación', 14),
 ('teatro', 12),
 ('continuación', 11),
 ('crónica', 10),
 ('parisiense', 10),
 ('cabello', 9),
 ('recomendado', 9),
 ('particular', 9),
 ('dentista', 9),
 ('viaje', 8),
 ('álbum', 8),
 ('día', 8),
 ('mujer', 7),
 ('primero', 7),
 ('señora', 7),
 ('madre', 7),
 ('abanico', 7),
 ('paráume', 7),
 ('joyas', 7),
 ('novela', 6),
 ('duelo', 6),
 ('cintas', 6),
 ('pasamanería', 6),
 ('fábrica', 6),
 ('amor', 5),
 ('obras', 5),
 ('salón', 5),
 ('niña', 4),
 ('costumbres', 4),
 ('hacer', 4),
 ('carta', 4),
 ('introducción', 4),
 ('costumbre', 4),
 ('baile', 4),
 ('secreto', 4),
 ('descripción', 4),
 ('cartas', 4),
 ('princpio', 4),
 ('fin', 4),
 ('moneda', 4),
 ('oro', 4),
 ('bretono', 3),
 ('Conclusión', 3),
 ('amigo', 3),
 ('lector', 3),
 ('curso', 3),
 ('familiar', 3),
 ('literatura', 3),
 ('inglés', 3),
 ('siguiente', 3),
 ('vestido', 3),
 ('emperador', 3),
 ('general', 

In [ ]:
with open("LEMA_FREQ.txt", "w") as f:
  for el in lema_freq.most_common():
    f.write(str(el)+"\n")

In [ ]:
ner_freq = Counter()

for element in pro_lem_cleaned:
  for titulo_ in element:
    for cat in titulo_:
      if isinstance(cat, str):
        if "[(" in cat:
          for match in re.finditer(r"\[\('([^']+)',\s*'[^']+'\)\]", cat):
          #for match in re.findinter(r"\[\('([^']+)',\s*'([^']+)'\)\]", cat): #group(2) the type of NER
            #print(match.group(0))
            #print(match.group(1))
            ##lema_freq.update([match.group(0)])
            ner_freq.update([match.group(1)])
            #lema_freq.update([cat[0]])

ner_freq.most_common()

[('Revista de Madrid', 18),
 ('Revista de París', 13),
 ('Platería Cristofle', 11),
 ('Revista Parisiense', 11),
 ('Camisas Claudio', 9),
 ('Hermanos', 9),
 ('Huéspedes', 9),
 ('Casa Delacroix', 9),
 ('Jules Bloch', 9),
 ('Nueva Geografía Sangansan', 8),
 ('Soneto', 7),
 ('Faguer Labollée', 7),
 ('Lemnnier', 7),
 ('Sorré Dehsle', 6),
 ('Corsets plastiques de Bonvallet', 6),
 ('Guantes', 6),
 ('Poesía', 5),
 ('doña Emilia Serrano de Wilson', 5),
 ('Revista Parisense', 5),
 ('Luna', 4),
 ('Cuba', 4),
 ('Tarifs des insertions', 4),
 ('Balada', 3),
 ('Margarita', 3),
 ('Paraguay', 3),
 ('Napoleón III', 3),
 ('Velázquez', 3),
 ('Fragmentos', 2),
 ('Margarita Aurora de Wilson', 2),
 ('Serrano', 2),
 ('Oda', 2),
 ('M. de Lamartine', 2),
 ('Brasil', 2),
 ('Leopoldo Robert', 2),
 ('Estados-Unidos', 2),
 ('Conclusión', 2),
 ('Memorándum especial por la guerra de Italia', 2),
 ('Solferino', 2),
 ('María Leczinska', 2),
 ('Griseldina', 2),
 ('Periquito el Veleidoso', 2),
 ('Santo Domingo', 2),
 ('

In [ ]:
with open("NER_FREQ.txt", "w") as f:
  for el in ner_freq.most_common():
    f.write(str(el)+"\n")

In [ ]:
lem_dic = dict(lema_freq.most_common())
lem_dic

{'revista': 65,
 'moda': 39,
 'mes': 19,
 'casa': 18,
 'histórico': 18,
 'figurín': 16,
 'explicación': 14,
 'teatro': 12,
 'continuación': 11,
 'crónica': 10,
 'parisiense': 10,
 'cabello': 9,
 'recomendado': 9,
 'particular': 9,
 'dentista': 9,
 'viaje': 8,
 'álbum': 8,
 'día': 8,
 'mujer': 7,
 'primero': 7,
 'señora': 7,
 'madre': 7,
 'abanico': 7,
 'paráume': 7,
 'joyas': 7,
 'novela': 6,
 'duelo': 6,
 'cintas': 6,
 'pasamanería': 6,
 'fábrica': 6,
 'amor': 5,
 'obras': 5,
 'salón': 5,
 'niña': 4,
 'costumbres': 4,
 'hacer': 4,
 'carta': 4,
 'introducción': 4,
 'costumbre': 4,
 'baile': 4,
 'secreto': 4,
 'descripción': 4,
 'cartas': 4,
 'princpio': 4,
 'fin': 4,
 'moneda': 4,
 'oro': 4,
 'bretono': 3,
 'Conclusión': 3,
 'amigo': 3,
 'lector': 3,
 'curso': 3,
 'familiar': 3,
 'literatura': 3,
 'inglés': 3,
 'siguiente': 3,
 'vestido': 3,
 'emperador': 3,
 'general': 3,
 'hermosura': 3,
 'hermana': 3,
 'cuadro': 3,
 'conuco': 3,
 'indio': 3,
 'contemporáneo': 3,
 'marquesa': 3,
 'Co

In [ ]:
ner_dic = dict(ner_freq.most_common())
ner_dic

{'Revista de Madrid': 18,
 'Revista de París': 13,
 'Platería Cristofle': 11,
 'Revista Parisiense': 11,
 'Camisas Claudio': 9,
 'Hermanos': 9,
 'Huéspedes': 9,
 'Casa Delacroix': 9,
 'Jules Bloch': 9,
 'Nueva Geografía Sangansan': 8,
 'Soneto': 7,
 'Faguer Labollée': 7,
 'Lemnnier': 7,
 'Sorré Dehsle': 6,
 'Corsets plastiques de Bonvallet': 6,
 'Guantes': 6,
 'Poesía': 5,
 'doña Emilia Serrano de Wilson': 5,
 'Revista Parisense': 5,
 'Luna': 4,
 'Cuba': 4,
 'Tarifs des insertions': 4,
 'Balada': 3,
 'Margarita': 3,
 'Paraguay': 3,
 'Napoleón III': 3,
 'Velázquez': 3,
 'Fragmentos': 2,
 'Margarita Aurora de Wilson': 2,
 'Serrano': 2,
 'Oda': 2,
 'M. de Lamartine': 2,
 'Brasil': 2,
 'Leopoldo Robert': 2,
 'Estados-Unidos': 2,
 'Conclusión': 2,
 'Memorándum especial por la guerra de Italia': 2,
 'Solferino': 2,
 'María Leczinska': 2,
 'Griseldina': 2,
 'Periquito el Veleidoso': 2,
 'Santo Domingo': 2,
 'Sevilla': 2,
 'Ucles': 2,
 'Rusia': 2,
 'Sombreros De Alexandrine': 2,
 'Bonvallet': 

In [32]:
pro_lem_cleaned[:20]

[[["[('Sr. D. R. P.', 'MISC')]",
   ('Enviando', 'enviar', 'VERB'),
   ('obras', 'obra', 'NOUN')]],
 [[('mujer', 'mujer', 'NOUN'),
   ('piadosa', 'piadoso', 'ADJ'),
   "[('Fragmentos', 'MISC')]"]],
 [["[('Desvarío', 'MISC')]"]],
 [["[('Soneto', 'PER')]",
   ('Gloria', 'gloria', 'PROPN'),
   ('pobreza', 'pobreza', 'NOUN'),
   "[('inmortal Quintana...', 'MISC')]"]],
 [["[('Balada', 'PER')]",
   ('despedida', 'despedida', 'NOUN'),
   ('hija', 'hija', 'NOUN')]],
 [[('viaje', 'viaje', 'NOUN'),
   "[('Luna', 'MISC')]",
   ('Prólogo', 'prólogo', 'NOUN'),
   ('pesadilla', 'pesadilla', 'PROPN'),
   "[('Mocquet', 'PER')]"]],
 [[('álbum', 'álbum', 'NOUN'), "[('señora M…', 'PER')]"]],
 [[('primavera', 'primavera', 'NOUN')]],
 [[('perejilera', 'perejilera', 'NOUN')]],
 [[('viaje', 'viaje', 'NOUN'),
   "[('Luna', 'MISC')]",
   ('Continuación', 'continuación', 'NOUN')]],
 [[('señor', 'señor', 'NOUN'), "[('Don B. M.', 'PER')]"]],
 [[('orillas', 'orilla', 'NOUN'),
   "[('Manzanares', 'LOC')]",
   ('súp

In [47]:
import re
from collections import Counter

all_results = []  # Final list: (ID_título, Categorías)

for idx, element in enumerate(pro_lem_cleaned[:388]):
  num_row = int(f"{idx+1}")

  # === Lemmas ===
  lemma_matches = []
  for titulo_ in element:
    for cat in titulo_:
      if isinstance(cat, tuple):
        form, lemma, pos = cat # We parse the structure to manipulate it
        lemma_matches.append((lemma, form))
        #lemma_pairs.append(f"{lemma}, {form}")

  lemma_counter = Counter(lemma_matches)
  for (lemma, form), freq in lemma_counter.items():
    all_results.append((num_row, lemma, form, 'LEMMA', freq))

  # === NERs ===
  ner_matches = []
  for titulo_ in element:
    for cat in titulo_:
      if isinstance(cat, str) and "[(" in cat:
        for match in re.finditer(r"\[\('([^']+)',\s*'[^']+'\)\]", cat):
          entity = match.group(1)
          ner_matches.append((entity, entity))  # we assume entity = form for now

  ner_counter = Counter(ner_matches)
  for (entity, form), freq in ner_counter.items():
    all_results.append((num_row, entity, form, 'NER', freq))

# === Preview ===
for row in all_results:
  print(row)


(1, 'enviar', 'Enviando', 'LEMMA', 1)
(1, 'obra', 'obras', 'LEMMA', 1)
(1, 'Sr. D. R. P.', 'Sr. D. R. P.', 'NER', 1)
(2, 'mujer', 'mujer', 'LEMMA', 1)
(2, 'piadoso', 'piadosa', 'LEMMA', 1)
(2, 'Fragmentos', 'Fragmentos', 'NER', 1)
(3, 'Desvarío', 'Desvarío', 'NER', 1)
(4, 'gloria', 'Gloria', 'LEMMA', 1)
(4, 'pobreza', 'pobreza', 'LEMMA', 1)
(4, 'Soneto', 'Soneto', 'NER', 1)
(4, 'inmortal Quintana...', 'inmortal Quintana...', 'NER', 1)
(5, 'despedida', 'despedida', 'LEMMA', 1)
(5, 'hija', 'hija', 'LEMMA', 1)
(5, 'Balada', 'Balada', 'NER', 1)
(6, 'viaje', 'viaje', 'LEMMA', 1)
(6, 'prólogo', 'Prólogo', 'LEMMA', 1)
(6, 'pesadilla', 'pesadilla', 'LEMMA', 1)
(6, 'Luna', 'Luna', 'NER', 1)
(6, 'Mocquet', 'Mocquet', 'NER', 1)
(7, 'álbum', 'álbum', 'LEMMA', 1)
(7, 'señora M…', 'señora M…', 'NER', 1)
(8, 'primavera', 'primavera', 'LEMMA', 1)
(9, 'perejilera', 'perejilera', 'LEMMA', 1)
(10, 'viaje', 'viaje', 'LEMMA', 1)
(10, 'continuación', 'Continuación', 'LEMMA', 1)
(10, 'Luna', 'Luna', 'NER', 1

In [48]:
with open("cate_DB.txt", "w") as f:
  for el in all_results:
    f.write(str(el)+"\n")

In [51]:
with open("CATEGORIAS_to_DB.csv", "w", newline="", encoding="utf-8") as csv_f:
  escribo = csv.writer(csv_f)
  escribo.writerow(["Num_row", "Categoría", "Form", "Tipo", "Frecuencia"])
  for row in all_results:
    escribo.writerow(row)